In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import zipfile

In [6]:
earnings = pd.read_excel('data/earnings formatted.xlsx')
earnings = earnings[earnings['Date'] > dt.datetime(2020,1,1)]
earnings = earnings.drop(earnings[earnings['Ticker'] == 'BRK'].index) # drop berkshire hathaway
earnings = earnings.drop(earnings[earnings['Ticker'] == 'BF'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'UHALB'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'LGF'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'METCV'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'GTXI'].index)
earnings

,Unnamed: 0,Ticker,Quarter,Date,Time
0,0,MSFT,2024:Q4,2024-07-30,16:00
1,1,MSFT,2024:Q3,2024-04-25,16:00
2,2,MSFT,2024:Q2,2024-01-30,16:00
3,3,MSFT,2024:Q1,2023-10-24,16:04
4,4,MSFT,2023:Q4,2023-07-25,16:00
...,...,...,...,...,...
182047,182047,WLLBW,2020:Q4,2021-02-24,17:34
182048,182048,WLLBW,2020:Q3,2020-11-05,06:02
182049,182049,WLLBW,2020:Q2,2020-08-05,08:00
182050,182050,WLLBW,2020:Q1,2020-05-18,09:00


In [7]:
returns = pd.read_csv('returns.csv.zip',compression='zip')
returns = returns.drop(returns[returns['Ticker'] == 'BRK'].index)
returns = returns.drop(returns[returns['Ticker'] == 'BF'].index)
returns = returns.drop(returns[returns['Ticker'] == 'UHALB'].index)
returns = returns.drop(returns[returns['Ticker'] == 'LGF'].index)
returns = returns.drop(returns[returns['Ticker'] == 'METCV'].index)
returns = returns.drop(returns[returns['Ticker'] == 'GTXI'].index)
returns

,Date,Ticker,Return
0,2020-01-03 00:00:00+00:00,A,-0.016056
1,2020-01-06 00:00:00+00:00,A,0.002956
2,2020-01-07 00:00:00+00:00,A,0.003065
3,2020-01-08 00:00:00+00:00,A,0.009873
4,2020-01-09 00:00:00+00:00,A,0.015712
...,...,...,...
2990267,2024-09-16 00:00:00+00:00,ZYXI,-0.022086
2990268,2024-09-17 00:00:00+00:00,ZYXI,0.027603
2990269,2024-09-18 00:00:00+00:00,ZYXI,-0.001221
2990270,2024-09-19 00:00:00+00:00,ZYXI,0.018337


In [9]:
#Disclaimer: idk if this code actually works, its based off sample.ipynb code - Andy
# Took 13:20 for this to finish running for me - Andy

ticker_list = returns['Ticker'].unique()

tickerdata_list = []

for ticker in ticker_list:
    ticker_data = yf.Ticker(ticker).history(start=dt.datetime(2020,1,1))
    ticker_data = ticker_data.reset_index()
    
    tickerdata_list.append(ticker_data)

final_df = pd.concat(tickerdata_list, keys=ticker_list).reset_index()
final_df = final_df.rename(columns={'level_0':'Ticker'})
final_df = final_df.drop(columns=['Open', 'High', 'Low', 'Dividends', 'Stock Splits', 'level_1'])
final_df.to_csv('yfinance.csv', index=False)

KeyboardInterrupt: 

In [ ]:
yfinance = pd.read_csv('yfinance.csv')
yfinance['Date'] = pd.to_datetime(yfinance['Date'])
yfinance['Date'] = yfinance['Date'].dt.tz_localize(None)
returns['Date'] = pd.to_datetime(returns['Date'])
returns['Date'] = returns['Date'].dt.tz_localize(None)

merge_list = ['Date', 'Ticker']
merged_df = returns.merge(yfinance, on=merge_list)

In [ ]:
ticker_list = merged_df['Ticker'].unique()

for ticker in ticker_list:
    ticker_data['log 1d ret'] = np.log1p(ticker_data['Return'])
    ticker_data['ret sum'] = ticker_data['log 1d ret'].cumsum()

    ticker_data['ret sum 3 shift'] = ticker_data['ret sum'].shift(-3)
    ticker_data['3d ret'] = ticker_data['ret sum 3 shift'] - ticker_data['ret sum']

    ticker_data['ret sum 7 shift'] = ticker_data['ret sum'].shift(-7)
    ticker_data['7d ret'] = ticker_data['ret sum 7 shift'] - ticker_data['ret sum']